In [1]:
import os
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [2]:
!nvidia-smi

Thu Sep 19 15:41:45 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     On  |   00000000:61:00.0 Off |                    0 |
|  0%   35C    P8             21W /  300W |       1MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Download image data using provided download CLI
Bellow we download all metadata and resized images (500p) for a FungiTastic-Mini subset.

In [4]:
!python ../../../dataset/download.py --metadata --images --subset "m" --size "500" --save_path "./"

In [5]:
!ls ./FungiTastic/metadata/FungiTastic-Mini/

FungiTastic-Mini-ClosedSet-Test.csv  FungiTastic-Mini-OpenSet-Test.csv
FungiTastic-Mini-ClosedSet-Val.csv   FungiTastic-Mini-OpenSet-Val.csv
FungiTastic-Mini-DNA-Test.csv	     FungiTastic-Mini-Train.csv


In [6]:
!ls ./FungiTastic/FungiTastic-Mini

dna-test  test	train  val


## Load and preprocess metadata

In [20]:
BASE_METADATA_PATH = "./"

train_df = pd.read_csv(f"{BASE_METADATA_PATH}/FungiTastic-Mini-Train.csv")
val_df = pd.read_csv(f"{BASE_METADATA_PATH}/FungiTastic-Mini-ClosedSet-Val.csv")
test_df = pd.read_csv(f"{BASE_METADATA_PATH}/FungiTastic-Mini-ClosedSet-Test.csv")

### Setting image_path to the images.

In [31]:
BASE_IMAGE_PATH = "./FungiTastic/FungiTastic-Mini"
SIZE = "500p"

train_df["image_path"] = train_df.filename.apply(lambda filename: f"{BASE_IMAGE_PATH}/train/{SIZE}/{filename}")
val_df["image_path"] = val_df.filename.apply(lambda filename: f"{BASE_IMAGE_PATH}/val/{SIZE}/{filename}")

### Update the column name to fit the FGVC expected naming convention and save the updated metadata

In [35]:
!mkdir metadata

train_df = train_df.rename(columns={"category_id":"class_id"})
val_df = val_df.rename(columns={"category_id":"class_id"})

train_df.to_csv("./metadata/FungiTastic-Mini-Train.csv", index=None)
val_df.to_csv("./metadata/FungiTastic-Mini-ClosedSet-Val.csv", index=None)

## Train single model with a config file.

### Environment variables
Select which GPU to use and add your WANDB_ENTITY and WANDB_PROJECT you want to log into etc.

In [36]:
%env CUDA_DEVICES = 0
%env WANDB_ENTITY = zcu_cv
%env HFHUB_OWNER = BVRA
%env WANDB_PROJECT = FungiTastic
%env TRAIN_METADATA_PATH = ./metadata/FungiTastic-Mini-Train.csv
%env VAL_METADATA_PATH = ./metadata/FungiTastic-Mini-ClosedSet-Val.csv
%env TEST_METADATA_PATH = ./metadata/FungiTastic-Mini-ClosedSet-Test.csv

os.environ["CUDA_VISIBLE_DEVICES"] = os.environ["CUDA_DEVICES"]

env: CUDA_DEVICES=0
env: WANDB_ENTITY=zcu_cv
env: HFHUB_OWNER=BVRA
env: WANDB_PROJECT=FungiTastic


In [45]:
!python train.py \
    --train-path "./metadata/FungiTastic-Mini-Train.csv" \
    --val-path "./metadata/FungiTastic-Mini-ClosedSet-Val.csv" \
    --test-path "./metadata/FungiTastic-Mini-ClosedSet-Test.csv" \
    --config-path ./configs/default_config.yaml \
    --cuda-devices $CUDA_DEVICES \
    --wandb-entity $WANDB_ENTITY \
    --wandb-project $WANDB_PROJECT \
    --hfhub-owner $HFHUB_OWNER

(script) INFO: Loading training config.
(script) DEBUG: Extra arguments passed to the script: {}
(script) INFO: Setting run name: resnet18.a1_in1k-CrossEntropyLoss-light
(script) INFO: Using experiment directory: ../runs/resnet18.a1_in1k-CrossEntropyLoss-light/exp12
(script) INFO: Using training configuration: {
    "augmentations": "light",
    "image_size": [
        224,
        224
    ],
    "dataset": "FungiTastic-Mini",
    "architecture": "resnet18.a1_in1k",
    "train": null,
    "loss": "CrossEntropyLoss",
    "optimizer": "SGD",
    "scheduler": "plateau",
    "epochs": 1,
    "learning_rate": 0.01,
    "batch_size": 64,
    "accumulation_steps": 1,
    "random_seed": 777,
    "workers": 12,
    "multigpu": false,
    "tags": [
        "FungiTastic-Mini",
        "224x224",
        "Convolutional"
    ],
    "root_path": "../",
    "run_name": "resnet18.a1_in1k-CrossEntropyLoss-light",
    "exp_name": "exp12",
    "exp_path": "../runs/resnet18.a1_in1k-CrossEntropyLoss-light/


KeyboardInterrupt



## Running Sweep Training

### Environment variables
Select which GPU to use and add your WANDB_ENTITY and WANDB_PROJECT you want to log into etc.

In [ ]:
%env CUDA_DEVICES = "changethis"
%env WANDB_ENTITY = "changethis"
%env HFHUB_OWNER = "changethis"
%env WANDB_PROJECT = "changethis"
%env TRAIN_METADATA_PATH = ./metadata/FungiTastic-Mini-Train.csv
%env VAL_METADATA_PATH = ./metadata/FungiTastic-Mini-ClosedSet-Val.csv
%env TEST_METADATA_PATH = ./metadata/FungiTastic-Mini-ClosedSet-Test.csv

os.environ["CUDA_VISIBLE_DEVICES"] = os.environ["CUDA_DEVICES"]

In [ ]:
!wandb sweep ./sweep_configs/FungiTastic-M-CloseSet-224.yaml

In [ ]:
!wandb agent <WANDB_ENTITY>/<WANDB_PROJECT>/.....